In [1]:
from utils.t5 import *
import warnings
import gc
torch.cuda.empty_cache()
pl.seed_everything(42)
warnings.filterwarnings("ignore")
gc.collect()
torch.cuda.empty_cache()

Global seed set to 42


In [2]:
input_model_name = "d-t5-fine-tunned_large"
input_data_name = "preprocessed.csv"  

input_model_dir = "./Data/Model/"
input_data_dir = "./Data/Preprocessed/"
output_dir = "./Data/Generation/"
source_column = "outline"           
generated_column = "automatic_text"       
model_type = "t5"

input_data_path = "{}{}".format(input_data_dir, input_data_name)
new_data_path = "{}generated_{}.pickle".format(output_dir,input_model_name)
input_model_path = "{}{}".format(input_model_dir,input_model_name)

In [3]:
data = pd.read_csv(input_data_path)

In [4]:
train, dev_test = train_test_split(data, test_size = 0.3, random_state = 42)
dev, test = train_test_split(dev_test, test_size = 0.5, random_state = 42)

In [5]:
model = SimpleT5()

In [6]:
input_model_path

'./Data/Model/d-t5-fine-tunned_large'

In [7]:
model.load_model(model_type, input_model_path, use_gpu = True)

In [8]:
input_data = test[source_column].to_list()
print(len(input_data))
automatic_lst = model.predict(input_data, batch_size = 15) # 30

45


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
new_data_path

'./Data/Generation/generated_d-t5-fine-tunned_large.pickle'

In [10]:
test[generated_column] = automatic_lst
print(len(test))
test.to_pickle(new_data_path)

45


### GPU

In [19]:
from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass

data = test[source_column].to_list()[0:100]

cpus = 4 #multiprocessing.cpu_count()


def m_predict(n):
    model.predict(n)

pool  = Pool(cpus)
automatic_lst = pool.map(m_predict, tqdm(data))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/100 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (749 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1328 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (728 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1284 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

In [ ]:
from torch.multiprocessing import set_start_method
set_start_method('spawn')

### CPU

In [ ]:
import multiprocessing

cpus = multiprocessing.cpu_count()
data = test[source_column].to_list()


def m_predict(n):
    model.predict(n)
    print("done")

pool = multiprocessing.Pool(processes=cpus)  # 
pool.map(m_predict, tqdm(data)) #automatic_lst = 

In [ ]:
#data = test[source_column].to_list()
#automatic_lst = [model.predict(i) for i in tqdm(data)]

In [ ]:
#test = test[["statement","label","explanation_prep","shortExplanation_prep"]]
test[generated_column] = automatic_lst
test.to_pickle(new_data_path)